In [1]:
ENV = 'local'  # 'colab'
if ENV == 'colab':
    !pip install -q PyDrive imbalanced-learn ipdb hyperopt

# Rete neurale per extreme returns su 2 azioni

Questo notebook contiene la parte di rete neurale per confronto con l'analisi statistica. Qui faremo *solo* l'ottimizzazione degli iperparametri, l'addestramento finale con i parametri ottimali trovati sarà fatta in un altro notebook.

Il flusso è il seguente:

- [x] utilizzo del dataset *S&P500* con la massima ampiezza storica disponibile (2005 - 2018)
- [x] calcolo dei log returns
- [x] selezione di due stocks, quelle con la minima e la massima volatilità in nel training set considerato
- [x] creazione estremi al 95%
- [x] oversampling con due possibili strategie: replicare le istanze positive, o replicarle con aggiunta di rumore gaussiano
- [x] addestramento rete con hyperparameter optimization
- [ ] ripetizione di ottimizzazione iperparametri per tutte e due le azioni con aggiunta di sentiment

Nell'altro notebook dovrò fare:
- [ ] utilizzo stesse metriche (ROC, KSS, Precision, Recall, Utility) che nel paper
- [ ] confronto con i risultati del modello probabilistico
- [ ] conclusioni

In [2]:
import os
import gc
import time
import datetime
from typing import Any, Dict, List, Tuple, Union
import pickle
import copy
import pprint
import uuid

import numpy as np
import pandas as pd
import pandas.testing as pt
import sklearn.metrics as sm
import sklearn.preprocessing as skpp
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from imblearn.over_sampling import RandomOverSampler
import hyperopt as hy
from hyperopt import hp, Trials, fmin, tpe, STATUS_OK

import keras

import matplotlib.pyplot as pl
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import seaborn as sns

import ipdb

%pdb on

Using TensorFlow backend.


Automatic pdb calling has been turned ON


In [3]:
if ENV == 'colab':
    from google.colab import drive
    drive.mount('/gdrive')

Un po' di dichiarazioni utili per il seguito

In [4]:
stock_type = ['min_vol', 'max_vol']
return_type = ['pos', 'neg', 'abs']
q_type = '95'
rs = 42  # random state
MAX_EPOCHS = 1000

stock_codes = {
    'min_vol': '9CE4C7',
    'max_vol': 'E28F22'
}  # già trovate in Paper-azioni.ipynb

stock_colors = {
    'min_vol': 'palegoldenrod',
    'max_vol': 'coral',
}

# i giorni sono i primi disponibili in quel mese nei dati
split_dates = {
    'subprime-crisis': datetime.datetime(2007, 1, 3), # subprime crisis
    'subprime-crisis-start': datetime.datetime(2007, 1, 3), # subprime crisis
    'subprime-crisis-halfway': datetime.datetime(2008, 9, 2),
    'subprime-crisis-end': datetime.datetime(2010, 1, 4),
    'eu-debt': datetime.datetime(2011, 1, 3), # EU sovereign debt crisis
    'eu-debt-halfway': datetime.datetime(2012, 1, 3), # EU sovereign debt crisis
    'last_train': datetime.datetime(2017, 1, 3), 
}

## 1. Importazione dei dati 

Per importare i dati dobbiamo caricarli, e poi usare la stategia "taglia-e-cuci" usata in `Paper-azioni.ipynb`.

In [6]:
if ENV == 'colab':
    data_path = '/gdrive/My Drive/OptiRisk Thesis/data'
    base_path = '/gdrive/My Drive/OptiRisk Thesis/experiments/11_final_experiment'
else:
    data_path = "/Users/pietro/Google Drive/OptiRisk Thesis/data"
    base_path = "/Users/pietro/Google Drive/OptiRisk Thesis/experiments/11_final_experiment"

prices_path = os.path.join(data_path, 'prices', 'adjusted_prices_volume.csv')
ta_dir = os.path.join(data_path, 'technical_features', 'features_all_years')
print(f"BASE path: {base_path}")
print(f"TA dir: {ta_dir}")

BASE path: /Users/pietro/Google Drive/OptiRisk Thesis/experiments/11_final_experiment
TA dir: /Users/pietro/Google Drive/OptiRisk Thesis/data/technical_features/features_all_years


Conversione delle date e settaggio dell'index del dataframe

In [7]:
prices = pd.read_csv(prices_path)
prices.loc[:, 'date'] = pd.to_datetime(prices['date'], format="%Y%m%d")
prices.index = prices['date']
prices.drop(columns=['date'], inplace=True)
prices.head()

,ravenpackId,open,low,high,close,volume
date,,,,,,
2005-01-03,00067A,29.86,28.97,29.86,29.16,784200.0
2005-01-04,00067A,29.36,29.03,29.59,29.10,1452100.0
2005-01-05,00067A,29.15,28.92,29.85,29.70,1564400.0
2005-01-06,00067A,29.75,29.61,30.30,29.64,1321600.0
2005-01-07,00067A,29.39,29.26,29.56,29.42,808400.0


Trasformiamola un una serie temporale, ogni riga una data, ogni colonna un'azione.

I prezzi:

In [8]:
prices_ts = prices.pivot(columns='ravenpackId', values='close')
prices_ts_no_nan = prices_ts.dropna(axis='columns', how='any', inplace=False)
prices_ts_no_nan.head()

ravenpackId,00067A,003B70,013528,0157B1,034B61,03B8CF,048590,055018,0624BE,067779,06EF42,071860,07CA6A,07EC43,095294,09F623,0A9D0A,0B57D7,0BC29E,0BE0AE,0BF4BA,0CE7F6,0E431C,0F0440,1151F4,12DE76,131443,143C52,147C38,1490F3,14A113,14BA06,14ED2B,159AE4,15ABD0,164D72,168A5D,16B183,1782D5,1791E7,...,EB6965,EC70EC,EC99D0,ECD263,ECF709,ED79D9,EE6F1C,EEA6B3,EF1AD9,EF5BED,EFD406,F11638,F1FA25,F30508,F39E1E,F40EE2,F4E882,F50BAB,F57F6F,F5C8AB,F5D410,F67165,F6DCE4,F6E248,F82D37,F93C8A,F97C81,FA40E2,FACF19,FAE021,FC1B7B,FC1F9D,FD39EB,FD5C77,FE7A63,FE89E0,FEC475,FEE4B0,FF4BA4,FF6644
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2005-01-03,29.16,25.02,51.94,44.52,23.73,82.40,5.520937,83.80,46.31,57.31,29.5125,19.190565,48.57,10.4575,20.253666,12.199479,16.988913,27.79,1106.385474,56.469823,12.45,26.53,23.24,49.27,18.02,19.32,20.5750,9.513338,33.06,13.59,26.17,37.20,22.795,28.750,23.190,32.125,57.980,48.28,12.733340,13.2700,...,25.010,16.623234,26.79,1.702855,90.75,16.089644,28.404721,20.770,46.66,35.096468,19.680,21.450,88.53,56.25,29.033333,11.71,19.17,32.275493,12.35,27.420,14.335,82.77,51.80,8.6825,17.25,14.049817,29.507073,22.745997,40.38,85.00,48.670100,9.2600,17.705,31.60,87.272161,32.059368,53.86,10.8450,24.777032,33.605482
2005-01-04,29.10,24.79,51.57,42.14,23.88,81.73,5.377500,82.25,47.28,57.05,28.2075,18.919827,48.19,10.3400,19.881972,11.993747,16.548941,26.79,1110.071750,55.707593,12.15,26.01,21.42,47.70,17.75,18.56,20.0925,9.210005,33.04,13.65,25.77,37.11,22.335,28.410,23.195,31.880,57.100,47.89,12.820006,13.0100,...,24.835,16.118615,25.80,1.665713,89.70,15.665169,27.148167,20.480,46.38,34.555870,19.755,20.745,87.70,55.50,28.840000,11.53,19.19,31.872050,12.15,26.965,14.150,81.02,51.24,8.4700,16.56,13.989818,29.470589,22.668179,40.09,83.92,47.626845,9.0575,17.355,31.23,86.506884,31.849730,53.47,10.8450,24.396131,33.005724
2005-01-05,29.70,23.53,51.63,41.77,23.59,80.73,5.456250,81.43,47.47,56.95,27.6000,18.282795,47.90,10.2450,19.794515,11.910579,16.418949,25.88,1128.503130,54.977800,12.25,25.99,21.21,48.08,17.55,18.57,19.2175,9.433338,32.84,13.20,25.07,36.60,21.910,27.200,22.790,31.980,57.200,47.73,12.600006,12.8000,...,24.275,15.805835,25.64,1.599998,90.29,15.658431,26.855286,20.385,46.21,34.608869,19.650,20.510,86.59,56.08,28.893333,11.40,18.89,30.738565,12.00,27.815,13.920,78.09,51.28,8.3525,16.36,13.904819,28.950680,22.442509,41.12,83.75,48.352587,8.9075,17.140,30.66,86.200773,32.086049,52.94,10.5875,24.499677,32.910524
2005-01-06,29.64,23.55,52.00,41.05,24.18,81.46,5.380312,82.45,50.12,57.36,28.1550,18.163352,47.70,10.1225,19.652397,11.928088,16.498944,25.69,1130.346268,54.977800,12.33,26.21,21.42,47.02,17.52,18.85,19.1300,9.366671,32.63,13.39,25.51,37.11,22.200,27.705,22.800,32.605,57.480,48.50,12.800006,12.9000,...,24.865,15.985162,25.18,1.578570,89.55,15.314808,27.242645,20.550,46.41,35.297867,19.630,20.380,86.84,56.53,28.866666,11.74,18.88,30.738565,12.05,27.955,14.000,77.70,48.50,8.4050,16.40,13.949819,29.032771,22.613707,41.14,83.66,48.225582,9.0875,17.310,30.30,85.680385,32.551064,53.32,10.6725,24.599525,32.691565
2005-01-07,29.42,23.40,52.45,42.32,25.01,82.30,5.200312,81.96,49.50,57.50,28.0950,18.338536,47.10,10.1300,19.685193,11.901824,16.248960,25.77,1132.356964,54.750753,12.64,26.15,21.07,46.80,17.42,18.72,18.7625,9.283338,32.78,13.26,25.76,37.01,22.395,27.730,22.695,31.765,58.378,48.19,12.566673,12.8525,...,25.040,15.972651,25.46,1.588570,89.73,15.301333,26.907249,20.580,46.42,35.308467,19.935,20.635,86.70,56.26,28.700000,11.88,18.77,30.748171,12.00,28.225,14.155,77.19,49.02,8.3875,16.19,13.914819,29.051014,22.504763,41.50,83.50,48.125793,8.8650,17.275,31.50,85.251830,32.345238,52.62,10.5900,24.984124,32.567805


I volumi:

In [9]:
volume_ts = prices.pivot(columns='ravenpackId', values='volume')
volume_ts_no_nan = volume_ts.loc[:, prices_ts_no_nan.columns]
pt.assert_index_equal(prices_ts_no_nan.columns, volume_ts_no_nan.columns, check_names=False)
volume_ts_no_nan.head()

ravenpackId,00067A,003B70,013528,0157B1,034B61,03B8CF,048590,055018,0624BE,067779,06EF42,071860,07CA6A,07EC43,095294,09F623,0A9D0A,0B57D7,0BC29E,0BE0AE,0BF4BA,0CE7F6,0E431C,0F0440,1151F4,12DE76,131443,143C52,147C38,1490F3,14A113,14BA06,14ED2B,159AE4,15ABD0,164D72,168A5D,16B183,1782D5,1791E7,...,EB6965,EC70EC,EC99D0,ECD263,ECF709,ED79D9,EE6F1C,EEA6B3,EF1AD9,EF5BED,EFD406,F11638,F1FA25,F30508,F39E1E,F40EE2,F4E882,F50BAB,F57F6F,F5C8AB,F5D410,F67165,F6DCE4,F6E248,F82D37,F93C8A,F97C81,FA40E2,FACF19,FAE021,FC1B7B,FC1F9D,FD39EB,FD5C77,FE7A63,FE89E0,FEC475,FEE4B0,FF4BA4,FF6644
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2005-01-03,784200.0,342400.0,3513100.0,10481801.0,1128420.0,2470500.0,2013090.0,1332100.0,1389500.0,1582700.0,1378900.0,2319253.0,761300.0,2370116.0,744300.0,504400.0,434010.0,2006126.0,6297800.0,3528900.0,21100.0,846600.0,4316178.0,3357500.0,876800.0,57030624.0,411700.0,2194007.0,1589600.0,178500.0,1047300.0,3532500.0,486000.0,340600.0,302300.0,4226800.0,11610.0,2504400.0,1654700.0,244400.0,...,1961300.0,1138600.0,4659817.0,1637955.0,851200.0,2234800.0,335700.0,5395500.0,715200.0,2061000.0,571900.0,2358324.0,337900.0,2579600.0,948100.0,4435700.0,2185800.0,579500.0,261800.0,607149.0,441300.0,238300.0,3654200.0,3143289.0,218500.0,222000.0,409900.0,1439800.0,9755500.0,1779000.0,1897800.0,582397.0,433600.0,1570500.0,599400.0,2501700.0,1890100.0,336900.0,914500.0,3347400.0
2005-01-04,1452100.0,365000.0,3282500.0,19511331.0,2967534.0,2871400.0,2108554.0,1164200.0,2069800.0,2997800.0,2111100.0,3149591.0,514000.0,2590269.0,887200.0,530700.0,466235.0,2682223.0,6323400.0,1537600.0,71800.0,692000.0,10986212.0,3032700.0,1218900.0,106359426.0,635200.0,1818470.0,2011700.0,137100.0,1092700.0,2542000.0,643700.0,346600.0,373200.0,3503600.0,15280.0,2857800.0,760300.0,338800.0,...,1675100.0,802200.0,7618972.0,2482076.0,600100.0,2449800.0,831000.0,6643100.0,687100.0,3148500.0,554800.0,2218273.0,209800.0,3518700.0,624000.0,5924000.0,1242500.0,667400.0,675000.0,618626.0,347900.0,216700.0,2081200.0,5035632.0,363800.0,179000.0,477000.0,1644100.0,6594800.0,2491400.0,3311300.0,307987.0,880500.0,1757700.0,589500.0,2231200.0,1665000.0,566500.0,1084500.0,4165600.0
2005-01-05,1564400.0,507400.0,3343500.0,8362057.0,887324.0,2484300.0,1556511.0,1233600.0,1762400.0,2376400.0,2104100.0,2489874.0,653100.0,1489251.0,735000.0,511500.0,215965.0,3422863.0,9202400.0,1417600.0,96900.0,546300.0,10074626.0,2559500.0,1426900.0,66139402.0,1524600.0,1912952.0,1497600.0,231400.0,1868000.0,3399400.0,1080800.0,589500.0,222300.0,3293400.0,27460.0,2713300.0,768100.0,241800.0,...,3642400.0,850500.0,4380475.0,1820097.0,691500.0,2420300.0,790000.0,5451800.0,322200.0,2161900.0,465600.0,1928287.0,310400.0,2999800.0,464100.0,5628500.0,1973900.0,888200.0,275000.0,1028227.0,610600.0,371000.0,2229200.0,10857632.0,290200.0,222900.0,709400.0,1895800.0,6704700.0,1790900.0,2148500.0,189450.0,526200.0,1409700.0,411700.0,3258900.0,1626500.0,563500.0,1155100.0,4733900.0
2005-01-06,1321600.0,317500.0,2961200.0,8704606.0,2636821.0,2388400.0,1524140.0,1102000.0,5392400.0,3279800.0,1921500.0,2717232.0,652100.0,1411730.0,965900.0,390400.0,290141.0,1979287.0,6467700.0,880300.0,154300.0,496100.0,7346534.0,3870600.0,1001800.0,63820315.0,794200.0,747931.0,2231400.0,73200.0,1616400.0,3671700.0,1371200.0,326400.0,161900.0,3616700.0,15860.0,3020400.0,882700.0,145300.0,...,2519200.0,704600.0,4811616.0,1181874.0,431600.0,1893000.0,561100.0,4025100.0,337100.0,2088300.0,486200.0,1697628.0,367200.0,2232500.0,578900.0,7545100.0,1065100.0,500200.0,204300.0,760314.0,368300.0,499100.0,11677600.0,3929724.0,283900.0,146100.0,380500.0,933100.0,4613000.0,1280600.0,1905500.0,264535.0,341300.0,1942900.0,496900.0,2827500.0,1355800.0,422200.0,776300.0,3475600.0
2005-01-07,808400.0,305600.0,3734800.0,9874272.0,2252683.0,2885900.0,2515670.0,1002400.0,2541600.0,2091700.0,1268700.0,2180680.0,607900.0,1242442.0,835800.0,258500.0,356248.0,2679310.0,5359200.0,555300.0,239800.0,

Ora calcoliamo i log-returns e le direzioni:

In [ ]:
log_returns = np.log(prices_ts_no_nan).diff(periods=1).iloc[1:, :]
directions_ts_no_nan = prices_ts_no_nan.diff(periods=1).iloc[1:, :]
prices_ts_no_nan = prices_ts_no_nan.iloc[1:, :]
volume_ts_no_nan = volume_ts_no_nan.iloc[1:, :]

pt.assert_index_equal(prices_ts_no_nan.index, volume_ts_no_nan.index)
pt.assert_index_equal(prices_ts_no_nan.index, log_returns.index)
pt.assert_index_equal(prices_ts_no_nan.index, directions_ts_no_nan.index)

Mi conviene creare una funzione che standardizzi le features, visto che poi ne avrò più di una (es: returns + volume).

In [ ]:
def only_train_notime(feature: pd.Series) -> pd.Series:
    """Just return the training part of a Series."""
    f = feature[np.logical_or(
        feature.index < split_dates['subprime-crisis-halfway'],
        np.logical_and(
            feature.index >= split_dates['eu-debt-halfway'],
            feature.index < split_dates['last_train']
        )
    )]

    return f

def standardize(feature: pd.Series) -> pd.Series:
    """Standardize a feature by computing the statistics on the training set."""
    # prendo solo la parte di training, perdendo ogni riferimento alla
    # sequenza temporale
    tmp_feature_train = only_train_notime(feature)

    scaler = skpp.RobustScaler()
    scaler.fit(tmp_feature_train.values.reshape(-1, 1))

    result = pd.Series(
        data=scaler.transform(feature.values.reshape(-1, 1)).flatten(),
        index=feature.index
    )
    
    return result

Ora creo i thresholds:

In [ ]:
# ora creo i dati per i returns (non standardizzati), i thresholds e i volumi (standardizzati)
lr_train_notime = dict()
lr_test_notime = dict()
returns_train_notime = dict()

# aggiungiamo i dati in modalità taglia-e-cuci
for s_type, s_code in stock_codes.items():
    # training set
    lr_current = log_returns.loc[:, s_code]
    lr_train_notime[s_type] = only_train_notime(lr_current)
    
    # returns train, tutti POSITIVI
    returns_train_notime[s_type] = {
        'pos': lr_train_notime[s_type][lr_train_notime[s_type] > 0.0],
        'neg': -(lr_train_notime[s_type][lr_train_notime[s_type] < 0.0]),
        'abs': lr_train_notime[s_type].abs()
    }

    

# ora creo i threshold
thresholds = {
    s_type: {
        ret_type: {
            q_type: returns_train_notime[s_type][ret_type].quantile(0.95)
        }
        for ret_type in return_type
    }
    for s_type in stock_type
}

ed infine creo i DataFrame e gli arrays che contengono tutti gli estremi e tutti i dati.

Le features che qui utilizziamo sono:

- log-returns standardizzati
- volume scambiato standardizzato
- tutte le features di TA che ci sono nel white paper di Douglas

In [13]:
feature_names = [
    'adx', 'aroon_down', 'aroon_up', 'atr', 'bb_lower', 'bb_middle', 'bb_upper',
    'cci', 'cmo', 'ema5', 'ema10', 'ema15', 'macd', 'rsi', 'sma5', 'sma10', 'sma15',
]

feature_paths = [os.path.join(ta_dir, name + '.h5') for name in feature_names]

features = dict()
first_allowable_dates = dict()  # date in cui posso prendere le feature e i returns

to_standardize = {
    'sma5', 'sma10', 'sma15',
    'ema5', 'ema10', 'ema15',
    'macd',
    'bb_lower', 'bb_middle', 'bb_upper',
    'roc', 'atr', 'cci', 'adx',
    }

to_divide = {
    'rsi': 100.0,
    'aroon_down': 100.0,
    'aroon_up': 100.0,
    'cmo': 100.0,
}

for s_type, s_code in stock_codes.items():
    print(f"Stock type: {s_type}")
    print("-"*30)
    features[s_type] = dict()

    for feature_name, feature_path in zip(feature_names, feature_paths):
        feature = pd.read_hdf(feature_path)

        if feature_name in to_standardize:
            print(f"standardizing {feature_name}")
            feature_transformed = standardize(feature.loc[:, s_code])
            features[s_type][feature_name] = feature_transformed
        elif feature_name in to_divide.keys():
            print(f"dividing {feature_name}")
            features[s_type][feature_name] = feature.loc[:, s_code] / to_divide[feature_name]
        else:
            raise ValueError(f"unknown feature {feature_name}")

    print("-" * 30)
    print("")

Stock type: min_vol
------------------------------
standardizing adx
dividing aroon_down
dividing aroon_up
standardizing atr
standardizing bb_lower
standardizing bb_middle
standardizing bb_upper
standardizing cci
dividing cmo
standardizing ema5
standardizing ema10
standardizing ema15
standardizing macd
dividing rsi
standardizing sma5
standardizing sma10
standardizing sma15
------------------------------

Stock type: max_vol
------------------------------
standardizing adx
dividing aroon_down
dividing aroon_up
standardizing atr
standardizing bb_lower
standardizing bb_middle
standardizing bb_upper
standardizing cci
dividing cmo
standardizing ema5
standardizing ema10
standardizing ema15
standardizing macd
dividing rsi
standardizing sma5
standardizing sma10
standardizing sma15
------------------------------



In [ ]:
extremes_all = dict()  # keys: s_type, q_type
data_all = dict()  # keys: s_type
volumes = dict()  # keys: s_type
directions_all = dict()  # keys: s_type

for s_type, s_code in stock_codes.items():
    # i returns
    lr = log_returns.loc[:, s_code]
    lr_transformed = standardize(lr)

    # i volumi
    stock_volume = volume_ts_no_nan.loc[:, s_code]
    volume_transformed = standardize(stock_volume)
    volumes[s_type] = volume_transformed

    # le features tecniche
    all_features = [lr_transformed, volume_transformed] + \
              [features[s_type][name] for name in feature_names]

    # tutte le features in un unico DataFrame
    tmp_df = pd.concat(
        all_features,
        axis=1,
        keys=['log_return', 'volume'] + feature_names
    )

    tmp_df = tmp_df.dropna(axis='index', how='any')
    
    data_all[s_type] = tmp_df
    extremes_all[s_type] = dict()
    
    ext = np.logical_or(
        lr >= thresholds[s_type]['pos'][q_type],
        lr <= -thresholds[s_type]['neg'][q_type],
    )
    
    extremes_all[s_type][q_type] = pd.Series(data=ext, index=log_returns.index)
    
    # le direzioni
    direction = (directions_ts_no_nan.loc[:, s_code] > 0.0).astype(np.int8)
    directions_all[s_type] = direction

## 2. Creazione dataset train-test per TensorFlow

Ora che ho i thresholds, posso creare il dataset vero e proprio, cioè:

- X: cubo dati
- y: estremo si/no

Per prima cosa, creo delle funzioni che mi creano i dati:

In [ ]:
# testata, funziona con array, Series e DataFrame
def rolling_window(data: np.ndarray,
                   start: int,
                   end: int,
                   lookback: int) -> np.ndarray:
    """
    Create a rolling window view of data, starting at index start, finishing
    at index end, with loockback days of bptt.
    
    Parameters
    ----------
    data: series, dataframe or array
        the data, containing one row for each time point and one column for each feature
        
    start: int
        starting index in the data
        
    end: int
        index where the whole thing ends, data[end] is **excluded**
        
    lookback: int
        length of the lookback period
        
    Returns
    -------
    X: np.ndarray
        array of shape(n_points, lookback, n_features)
    """
    assert lookback < data.shape[0]  # lookback sano
    assert start - lookback + 1 >= 0  # lookback sano
    
    n_features = data.shape[1]
    n_points = end - start
    
    X = np.zeros((n_points, lookback, n_features), dtype = data.dtype)
    
    # range strano per l'indicizzazione numpy
    for i, t in enumerate(range(start + 1, end + 1)):
        X[i, :, :] = data[t - lookback:t, :]
        
    return X


# testata, funziona hehehe
def rolling_window_xyd(data: Union[pd.Series, pd.DataFrame],
                      targets: List[pd.Series],
                      start: int,
                      end: int,
                      lookback: int) -> Tuple[np.ndarray, List[np.ndarray], pd.Series]:
    """
    Create X, y and dates in a single shot.
    The returned dates are relative to the y array(s).
    """
    if isinstance(data, pd.Series):
        my_data = data.values.reshape(-1, 1)
    elif isinstance(data, pd.DataFrame):
        my_data = data.values
    else:
        raise TypeError("data should be a pandas Series or Dataframe")

    X = rolling_window(my_data, start, end, lookback)
    
    if not isinstance(targets, list):
        raise TypeError("target must be a list of pandas Series")
    if not all(isinstance(t, pd.Series) for t in targets):
        raise TypeError("all targets should be pandas Series")
    if not all(isinstance(t.index, pd.DatetimeIndex) for t in targets):
        raise TypeError("index of target should be a pandas DatetimeIndex")
        
    y = [t.values[start + 1:end + 1] for t in targets]
    dates = pd.Series(data=targets[0].index[start + 1: end + 1])
        
    return X, y, dates


# TESTATO: funziona
def create_Xyd(returns: Union[pd.Series, pd.DataFrame],
               extremes: pd.Series,
               directions: pd.Series,
               lookback: int) -> Tuple[
    np.ndarray, np.ndarray, List[np.ndarray], List[np.ndarray], pd.Series, pd.Series
]:
    """
    Create the X, y and dates arrays for the ANN.
    """
    test_start_1 = returns.index.get_loc(split_dates['subprime-crisis-halfway'])
    test_end_1 = returns.index.get_loc(split_dates['eu-debt-halfway'])
    test_start_2 = returns.index.get_loc(split_dates['last_train'])

    # TRAIN
    tmp_X_train_1, tmp_y_train_1, tmp_dates_train_1 = rolling_window_xyd(
        returns,
        [extremes, directions],
        start=lookback - 1,  # sempre lookback - 1 se il primo iniziale
        end=test_start_1,
        lookback=lookback
    )

    tmp_X_train_2, tmp_y_train_2, tmp_dates_train_2 = rolling_window_xyd(
        returns,
        [extremes, directions],
        start=test_end_1,  # sempre lookback - 1 se il primo iniziale
        end=test_start_2,
        lookback=lookback
    )
    
    assert len(tmp_y_train_1) == len(tmp_y_train_2)
    
    X_train = np.concatenate([tmp_X_train_1, tmp_X_train_2])
    y_train = [np.concatenate([tmp_y_train_1[i], tmp_y_train_2[i]]) for i in range(len(tmp_y_train_1))]
    dates_train = pd.concat([tmp_dates_train_1, tmp_dates_train_2], axis=0, ignore_index=True).values
    assert X_train.shape[0] == dates_train.shape[0]
    assert all(yy.shape[0] == X_train.shape[0] for yy in y_train)

    # TEST
    tmp_X_test_1, tmp_y_test_1, tmp_dates_test_1 = rolling_window_xyd(
        returns,
        [extremes, directions],
        start=test_start_1,  # sempre lookback - 1 se il primo iniziale
        end=test_end_1,
        lookback=lookback
    )
    
    tmp_X_test_2, tmp_y_test_2, tmp_dates_test_2 = rolling_window_xyd(
        returns,
        [extremes, directions],
        start=test_start_2,  # sempre lookback - 1 se il primo iniziale
        end=returns.shape[0] - 1,
        lookback=lookback
    )
  
    X_test = np.concatenate([tmp_X_test_1, tmp_X_test_2])
    y_test = [np.concatenate([tmp_y_test_1[i], tmp_y_test_2[i]]) for i in range(len(tmp_y_test_1))]
    dates_test = pd.concat([tmp_dates_test_1, tmp_dates_test_2], axis=0, ignore_index=True).values
    assert X_test.shape[0] == dates_test.shape[0]
    assert all(yy.shape[0] == X_test.shape[0] for yy in y_test)

    return X_train, X_test, y_train, y_test, dates_train, dates_test


def split_stratified(X: np.ndarray,
                     y: List[np.ndarray],
                     dates: np.ndarray,
                     test_size=0.2,
                     random_state=rs,
                     verbose=False):
    """
    Split a dataset in a stratified fashion on the target variable y[0].
    """
    assert X.ndim == 3
    # divido in train-validation, lo faccio prendendo gli indici dagli estremi y/n con un
    # ShuffleSplit che divide a caso
    splitter = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    
    n_samples = X.shape[0]
    n_features = X.shape[2]
    
    XX = np.zeros(n_samples, dtype=np.int8)
    
    if verbose:
        for i in range(len(y)):
            vals, counts = np.unique(y[i], return_counts=True)
            for v, c in zip(vals, counts):
                print(f"y[{i}] has {c} elements of class {v}")
    
    train_index, test_index = next(splitter.split(XX, y[0]))
    
    X_train = X[train_index]
    X_validation = X[test_index]
    
    y_train = [yy[train_index] for yy in y]
    y_validation = [yy[test_index] for yy in y]
    
    dates_train = dates[train_index]
    dates_validation = dates[test_index]

    return X_train, X_validation, y_train, y_validation, dates_train, dates_validation


def oversample_mtl(X: np.ndarray, y: List[np.ndarray], random_state=rs, dt=np.float32):
    """Oversample a dataset on the positive 1 class."""
    assert X.dtype == dt
    assert X.ndim == 3
    assert isinstance(y, list) and all(yy.ndim == 1 for yy in y) and all(yy.dtype == dt for yy in y)
    
    # oversample
    ro = RandomOverSampler(random_state=random_state)
    nx = X.shape[0]
    indexes = np.arange(nx).reshape(nx, 1)
    
    indexes_resampled, y_resampled = ro.fit_resample(indexes, y[0])
    ir = indexes_resampled.flatten()
    
    X_resampled = X[ir]
    y_resampled = [yy[ir] for yy in y]
    
    return X_resampled, y_resampled

## 3. Addestramento rete

Visto che serve l'ottimizzazione degli iperparametri, è importante avere una funzione da ottimizzare con Hyperopt. Aggiungiamo le funzioni che servono.

La prima crea il modello:

In [ ]:
def create_model_mtl(space: Dict[str, Any],
                     bptt: int,
                     n_features: int) -> keras.models.Model:
    """Create a model using the parameters in the search space."""
    l = space['layers']

    input_dropout = float(l['input_dropout'])
    assert input_dropout >= 0.0 and input_dropout <= 1.0

    n_layers = int(l['num_layers']['how_many'])
    assert n_layers <= 2 and n_layers > 0

#     n_cells_1 = int(l['num_layers']['n_cells_1'])
#     assert n_cells_1 >= 1

    # creo il modello
    model_input = keras.Input(shape=(bptt, n_features), name='model_input')

    if n_layers == 1:
        if input_dropout > 0.0:
            x = keras.layers.LSTM(n_features, dropout=input_dropout)(model_input)
        else:
            x = keras.layers.LSTM(n_features)(model_input)
    elif n_layers == 2:
        n_cells_2 = int(l['num_layers']['n_cells_2'])
        x = keras.layers.LSTM(n_features, return_sequences=True)(model_input)
        x = keras.layers.LSTM(n_cells_2)(x)
    elif n_layers == 3:
        n_cells_2 = int(l['num_layers']['n_cells_2'])
        n_cells_3 = int(l['num_layers']['n_cells_3'])
        x = keras.layers.LSTM(n_features, return_sequences=True)(model_input)
        x = keras.layers.LSTM(n_cells_2, return_sequences=True)(x)
        x = keras.layers.LSTM(n_cells_3)(x)

    output_is_extreme = keras.layers.Dense(
        2, activation='softmax', name='extreme')(x)
    output_is_up_down = keras.layers.Dense(
        2, activation='softmax', name='up_down')(x)

    model = keras.Model(
        inputs=model_input,
        outputs=[output_is_extreme, output_is_up_down],
        name='MTL_model')

    return model

La seconda valuta le performance e trova i TP, FP, TN, FN:

In [ ]:
def get_tptnfpfn(y_true, y_pred, labels=[0, 1]) -> Dict[str, int]:
    """Get the tp, tn, fp, fn count for y_true and y_pred."""
    cm = sm.confusion_matrix(y_true, y_pred, labels=labels)
    tp = cm[1, 1]
    tn = cm[0, 0]
    fp = cm[0, 1]
    fn = cm[1, 0]

    return {
        'tp': tp,
        'fp': fp,
        'tn': tn,
        'fn': fn,
    }


def compute_performance(tp, tn, fp, fn, weighted=False):
    support_1 = tp + fn
    support_0 = tn + fp
    
    if tp + fp > 0:
        prec_1 = tp / (tp + fp)
    else:
        prec_1 = np.nan
        
    if tn + fn > 0:
        prec_0 = tn / (tn + fn)
    else:
        prec_0 = np.nan
    
    rec_1 = tp / (tp + fn)
    rec_0 = tn / (tn + fp)
    
    if any(np.isnan(x) for x in [prec_1, prec_0, rec_1, rec_0]):
        return np.nan
    else:
        f1_1 = 2.0 * (prec_1 * rec_1) / (prec_1 + rec_1)
        f1_0 = 2.0 * (prec_0 * rec_0) / (prec_0 + rec_0)
        
        if weighted: # pesa di più la classe di maggioranza
            f1_tot = np.average([f1_0, f1_1], weights=[support_0, support_1])
        else:
            f1_tot = np.mean([f1_0, f1_1])
            
        return f1_tot

La terza mi crea il dataframe dei risultati che poi posso analizzare:

In [ ]:
def get_new_results() -> pd.DataFrame:
    return pd.DataFrame(
        data=None,
        columns=[
            'dataset',
            'optimizer',
            'start_time',
            'experiment_id',
            'trial',
            'bptt',
            'lr',
            'n_layers',
            'n_cells_1',
            'n_cells_2',
            'n_cells_3',
            'dropout',
            'loss_extreme',
            'loss_up_down',
            'loss_volume',
            'f1_validation_extreme',
            'f1_validation_up_down',
            'tp_extreme',
            'tn_extreme',
            'fp_extreme',
            'fn_extreme',
            'tp_up_down',
            'tn_up_down',
            'fp_up_down',
            'fn_up_down',
            'train_epochs',
            'es_min_delta',
            'es_patience',
            'use_class_weight',
            'output',
        ])

Infine l'ultima esegue l'esperimento.

In [ ]:
def run_experiment(space,
                   stock_type: str,
                   max_epochs: int,
                   data: Union[pd.Series, pd.DataFrame],
                   extremes: pd.Series,
                   directions: pd.Series,
                   n_runs: int,
                   verbose: int,
                   results_path: str):
    """Fai girare un singolo esperimento.
    
    Parameters
    ----------
    space: 
        hyperopt search space
        
    stock_type:
        type of stock to use
    
    max_epochs: int
        number of max epochs to tun the model for
        
    data: pd.Series of shape (n_timepoints,), or pd.DataFrame of shape (n_timepoints, n_features)
        data containing returns, volume and all other things, where every row is
        a timepoint and every column a different feature
        
    extremes: pd.Series of shape (n_timepoints,)
        target for the extremes, binary 1/0, 
        
    directions: pd.Series of shape (n_timepoints,)
        target for the directions
        
    n_runs: int
        how many times to run a model with the same structure to get a reliable
        estimate of the loss function
        
    verbose: int
        verbosity for Keras
        
    results_path: str or path
        path where to save the intermediate runs
    """
    sigmoid_or_softmax = 'softmax'
    if data.ndim == 1:
        n_features = 1
    else:
        n_features = data.shape[1]

    lookback = bptt = int(space['bptt'])
    batch_size = data.shape[0]
    
    # 1. creazione dataset per questo lookback
    X_trv, X_test, y_trv, y_test, dates_trv, dates_test = create_Xyd(
        data.astype(np.float32),
        extremes.astype(np.float32),
        directions.astype(np.float32),
        lookback=lookback
    )

    # divido in train-validation
    X_train, X_validation, y_train, y_validation, dates_train, dates_validation = split_stratified(
        X_trv,
        y_trv,
        dates_trv,
        test_size=0.2,
        verbose=True
    )

    # bilancio con oversampling della classe di minoranza (1)
    X_train_bal, y_train_bal = oversample_mtl(X_train, y_train)  # bal = balanced
    
    # 2. creo il file dei risultati
    if not os.path.exists(results_path):
        print(f"Creating new result file at {results_path}")
        results = get_new_results()
    else:
        print(f"Loading result file at {results_path}")
        results = pd.read_csv(results_path)

    # 3. creo le variabili che servono per il training (dati e parametri)
    print(space)
    try:
        use_class_weight = space['use_class_weight']
        if use_class_weight:
            print("Using class weight for training")
    except KeyError:
        use_class_weight = False
        
    
    y_train_bal_cat = [keras.utils.to_categorical(yy, num_classes=2) for yy in y_train_bal]
    y_validation_cat = [keras.utils.to_categorical(yy, num_classes=2) for yy in y_validation]
    y_test_cat = [keras.utils.to_categorical(yy, num_classes=2) for yy in y_test]

    # 4. inizializza le loss a 0 e crea i tempi di inizio e l'id esperimento
    train_losses: List[float] = []
    val_losses = []
    test_losses = []

    optimizer_name = space['optimizer']['name']
    assert optimizer_name in {'adam', 'adadelta'}

    start_time = int(round(time.time()))
    experiment_id = str(uuid.uuid4())

    # 5. addestra e testa il modello per n_runs volte
    for i in range(n_runs):
        model = create_model_mtl(space, lookback, n_features)

        # 5.1 crea l'optimizer
        if optimizer_name == 'adam':
            learning_rate = space['optimizer']['lr']
            optimizer = keras.optimizers.Adam(lr=learning_rate)
        elif optimizer_name == 'adadelta':
            optimizer = 'adadelta'
            learning_rate = 1.0
        else:
            raise ValueError(f"Invalid optimizer name {optimizer_name}")

        if i == 0:
            print(
                f"Hyper parameters: bptt={bptt}, optimizer={optimizer_name}, learning_rate={learning_rate}"
            )
            # model.summary()

        # 5.2 compila il modello
        model.compile(
            optimizer=optimizer,
            loss=['categorical_crossentropy', 'categorical_crossentropy'],
        )

        # 5.3 parametri per l'Early Stopping
        min_delta = float(space['early_stop']['min_delta'])
        patience = int(space['early_stop']['patience'])

        early_stop_cb = keras.callbacks.EarlyStopping(
            monitor='val_loss',
            min_delta=min_delta,
            patience=patience,
            restore_best_weights=True)

        print(f"Iteration {i}")
        print("Fitting model")

#         if use_class_weight:
#             print(f"Class weights: {class_weights}")
#             history: keras.callbacks.History = model.fit(
#                 x=X_train,
#                 y=y_t,
#                 epochs=max_epochs,
#                 batch_size=batch_size,
#                 validation_data=(X_validation, y_v),
#                 callbacks=[early_stop_cb],
#                 shuffle=True,
#                 verbose=verbose,
#                 class_weight=class_weights)
#         else:
#             history: keras.callbacks.History = model.fit(  # type: ignore
#                 x=X_train,
#                 y=y_t,
#                 epochs=max_epochs,
#                 batch_size=batch_size,
#                 validation_data=(X_validation, y_v),
#                 callbacks=[early_stop_cb],
#                 shuffle=True,
#                 verbose=verbose)

        # 5.4 addestra il modello
        history: keras.callbacks.History = model.fit(  # type: ignore
            x=X_train_bal,
            y=y_train_bal_cat,
            epochs=max_epochs,
            batch_size=batch_size,
            validation_data=(X_validation, y_validation_cat),
            callbacks=[early_stop_cb],
            shuffle=True,
            verbose=verbose
        )

        # 5.5 valutalo su training e validation
        train_epochs = len(history.history['loss'])

        min_index = np.argmin(history.history['val_loss'])

        curr_train_loss_total = history.history['loss'][min_index]
        curr_train_loss_extreme = history.history['extreme_loss'][min_index]
        curr_train_loss_up_down = history.history['up_down_loss'][min_index]
        train_losses.append(curr_train_loss_total)

        curr_validation_loss_total = history.history['val_loss'][min_index]
        curr_validation_loss_extreme = history.history['val_extreme_loss'][
            min_index]
        curr_validation_loss_up_down = history.history['val_up_down_loss'][
            min_index]

        # 5.6 valutalo su test set
        current_test_loss = model.evaluate(X_test, y_test_cat, batch_size=batch_size, verbose=0)

        curr_test_loss_total, curr_test_loss_extreme, curr_test_loss_up_down = current_test_loss
        test_losses.append(curr_test_loss_total)

        # 5.7 salva train, validation e test set performance
        # train
        print("Evaluating the model on the training set")
        probs = model.predict(X_train_bal, batch_size=batch_size)
        y_pred_extreme, y_pred_up_down = [np.argmax(p, axis=1) for p in probs]
        
        y_true_extreme, y_true_up_down = y_train_bal

        r_extreme = get_tptnfpfn(y_true_extreme, y_pred_extreme)
        r_up_down = get_tptnfpfn(y_true_up_down, y_pred_up_down)

        row = pd.Series({
            'dataset': 'train',
            'optimizer': optimizer_name,
            'start_time': start_time,
            'experiment_id': experiment_id,
            'trial': i,
            'bptt': bptt,
            'lr': learning_rate,
            'n_layers': space['layers']['num_layers']['how_many'],
            'n_cells_1': space['layers']['num_layers']['n_cells_1'],
            'n_cells_2': space['layers']['num_layers']['n_cells_2'],
            'dropout': space['layers']['input_dropout'],
            'loss_extreme': curr_train_loss_extreme,
            'loss_up_down': curr_train_loss_up_down,
            'loss_volume': np.nan,
            'f1_validation_extreme': np.nan,
            'f1_validation_up_down': np.nan,
            'tp_extreme': r_extreme['tp'],
            'tn_extreme': r_extreme['tn'],
            'fp_extreme': r_extreme['fp'],
            'fn_extreme': r_extreme['fn'],
            'tp_up_down': r_up_down['tp'],
            'tn_up_down': r_up_down['tn'],
            'fp_up_down': r_up_down['fp'],
            'fn_up_down': r_up_down['fn'],
            'train_epochs': train_epochs,
            'es_min_delta': min_delta,
            'es_patience': patience,
            'use_class_weight': use_class_weight,
            'output': sigmoid_or_softmax,
        })
        results = results.append(row, ignore_index=True)

        # validation
        print("Evaluating the model on the validation set")
        probs = model.predict(X_validation, batch_size=batch_size)
        y_pred_extreme, y_pred_up_down = [np.argmax(p, axis=1) for p in probs]
        
        y_true_extreme, y_true_up_down = y_validation
        
        r_extreme = get_tptnfpfn(y_true_extreme, y_pred_extreme)
        r_up_down = get_tptnfpfn(y_true_up_down, y_pred_up_down)

        # ora voglio ottimizzare rispetto all'F1 score della classe +1 -->
        # utilizzo questo come metrica
        f1_extreme = compute_performance(r_extreme['tp'], r_extreme['tn'],
                                         r_extreme['fp'], r_extreme['fn'])
        f1_up_down = compute_performance(r_up_down['tp'], r_up_down['tn'],
                                         r_up_down['fp'], r_up_down['fn'])

#         # se l'f1 è zero, allora l'esperimento è fallito
#         if np.isnan(f1_extreme) or np.isnan(f1_up_down):
#             return {
#                 'status': STATUS_FAIL,
#                 'loss': 1e+30,
#             }

        val_losses.append(curr_validation_loss_total)

        row = pd.Series({
            'dataset': 'validation',
            'optimizer': optimizer_name,
            'start_time': start_time,
            'experiment_id': experiment_id,
            'trial': i,
            'bptt': bptt,
            'lr': learning_rate,
            'n_layers': space['layers']['num_layers']['how_many'],
            'n_cells_1': space['layers']['num_layers']['n_cells_1'],
            'n_cells_2': space['layers']['num_layers']['n_cells_2'],
            'dropout': space['layers']['input_dropout'],
            'loss_extreme': curr_validation_loss_extreme,
            'loss_up_down': curr_validation_loss_up_down,
            'loss_volume': np.nan,
            'f1_validation_extreme': f1_extreme,
            'f1_validation_up_down': f1_up_down,
            'tp_extreme': r_extreme['tp'],
            'tn_extreme': r_extreme['tn'],
            'fp_extreme': r_extreme['fp'],
            'fn_extreme': r_extreme['fn'],
            'tp_up_down': r_up_down['tp'],
            'tn_up_down': r_up_down['tn'],
            'fp_up_down': r_up_down['fp'],
            'fn_up_down': r_up_down['fn'],
            'train_epochs': train_epochs,
            'es_min_delta': min_delta,
            'es_patience': patience,
            'use_class_weight': use_class_weight,
            'output': sigmoid_or_softmax,
        })
        results = results.append(row, ignore_index=True)

        # testing
        print("Evaluating the model on the test set")
        probs = model.predict(X_test, batch_size=batch_size)
        y_pred_extreme, y_pred_up_down = [np.argmax(p, axis=1) for p in probs]
        
        y_true_extreme, y_true_up_down = y_test

        r_extreme = get_tptnfpfn(y_true_extreme, y_pred_extreme)
        r_up_down = get_tptnfpfn(y_true_up_down, y_pred_up_down)

        row = pd.Series({
            'dataset': 'test',
            'optimizer': optimizer_name,
            'start_time': start_time,
            'experiment_id': experiment_id,
            'trial': i,
            'bptt': bptt,
            'lr': learning_rate,
            'n_layers': space['layers']['num_layers']['how_many'],
            'n_cells_1': space['layers']['num_layers']['n_cells_1'],
            'n_cells_2': space['layers']['num_layers']['n_cells_2'],
            'dropout': space['layers']['input_dropout'],
            'loss_extreme': curr_test_loss_extreme,
            'loss_up_down': curr_test_loss_up_down,
            'loss_volume': np.nan,
            'f1_validation_extreme': np.nan,
            'f1_validation_up_down': np.nan,
            'tp_extreme': r_extreme['tp'],
            'tn_extreme': r_extreme['tn'],
            'fp_extreme': r_extreme['fp'],
            'fn_extreme': r_extreme['fn'],
            'tp_up_down': r_up_down['tp'],
            'tn_up_down': r_up_down['tn'],
            'fp_up_down': r_up_down['fp'],
            'fn_up_down': r_up_down['fn'],
            'train_epochs': train_epochs,
            'es_min_delta': min_delta,
            'es_patience': patience,
            'use_class_weight': use_class_weight,
            'output': sigmoid_or_softmax,
        })
        results = results.append(row, ignore_index=True)

        # scrivo sul CSV
        print("Scrivo i risultati")
        results.to_csv(results_path, index=None)

        # cancello la memoria
        keras.backend.clear_session()
        gc.collect()

    # 6. media delle loss
    train_loss_mean = np.mean(train_losses)
    train_loss_variance = np.var(train_losses)

    val_loss_mean = np.mean(val_losses)
    val_loss_variance = np.var(val_losses)

    test_loss_mean = np.mean(test_losses)
    test_loss_variance = np.var(test_losses)

    # 7. return del dict di loss
    return {
        'status': STATUS_OK,
        'loss': val_loss_mean,
        'loss_variance': val_loss_variance,
        'true_loss': test_loss_mean,
        'true_loss_variance': test_loss_variance,
        'train_loss_mean': train_loss_mean,
        'train_loss_variance': train_loss_variance,
    }

### 3.1 Azione con minima volatilità - ottimizzazione iperparametri

Cominciamo con l'azione meno volatile. Per prima cosa, bisogna trovare la struttura migliore della rete, quindi usiamo l'ottimizzazione degli iperparametri per farlo.

In [ ]:
s_type = 'min_vol'
results_path = os.path.join(base_path, 'results', f"results_{s_type}_no_sentiment.csv")
print(f"RESULTS path: {results_path}")

def objective_fn(space, **kwargs):
    return run_experiment(
        space,
        stock_type=s_type,
        max_epochs=MAX_EPOCHS,
        data=data_all[s_type],
        extremes=extremes_all[s_type]['95'],
        directions=directions_all[s_type],
        n_runs=5,
        verbose=2,
        results_path=results_path
    )

Ora creo lo spazio di ricerca per questa azione:

In [ ]:
optimizer_space = hp.choice('opt_name', [
    {
        'name': 'adam',
        'lr': hp.uniform('lr_adam', low=1e-5, high=1e-2)
    },
   {
       'name': 'adadelta',
   },
])

layer_space = {
    'num_layers': {
        'how_many': 1,
        'n_cells_1': 0, # hp.quniform('number_of_cells', low=10, high=100, q=2), # 96
        'n_cells_2': 0,
    },
#     hp.choice('number_of_layers', [
#         {
#             'how_many': 1,
#             'n_cells_1': 0, # hp.quniform('number_of_cells', low=10, high=100, q=2), # 96
#             'n_cells_2': 0,
#         },
#        {
#            'how_many': 2,
#            'n_cells_1': hp.quniform('number_of_cells_1', low=10, high=100, q=2),
#            'n_cells_2': hp.quniform('number_of_cells_2', low=10, high=50, q=2),
#        }
#     ]),
    'input_dropout': hp.uniform('dropout_kill_rate', low=0.0, high=0.4),
}

early_stop_space = {
    'patience': hp.quniform('early_stop_patience', low=5, high=25, q=1),
    'min_delta': hp.quniform('early_stop_min_delta', low=1e-4, high=1e-2, q=2e-4)
}

opt_space = {
    'optimizer': optimizer_space,
    'layers': layer_space,
    'bptt': hp.quniform('bptt_len', low=10, high=120, q=1),
    'early_stop': early_stop_space,
    'use_class_weight': False,
}

Ora facciamo la ricerca degli iperparametri:

In [ ]:
print("COMINCIAMO\n")
n_trials = 50

load_trials = True
trials_filename = os.path.join(base_path, 'results', f"trials_{s_type}_no_sentiment.pickle")
best_filename = os.path.join(base_path, 'results', f"best_{s_type}_no_sentiment.pickle")

if load_trials:
    print("Loading trials from last execution")
    try:
        with open(trials_filename, 'rb') as infile:
            trials = pickle.load(infile)
            print(f"len(trials) = {len(trials)}")
    except FileNotFoundError:
        print("Trials file not found, starting with new trial object")
        trials = hy.Trials()
else:
    print("Starting new experiment with new trials object")
    trials = hy.Trials()


best = fmin(
    objective_fn,
    opt_space,
    algo=tpe.suggest,
    max_evals=len(trials) + n_trials,
    trials=trials
)

with open(trials_filename, 'wb') as outfile:
    pickle.dump(trials, outfile)

with open(best_filename, 'wb') as outfile:
    pickle.dump(best, outfile)

print("FINITO")

### 3.3 Azione massima volatilità - ottimizzazione iperparametri

Ora, facciamo la stessa cosa per l'azione con più volatilità. Ricorda che qui stiamo solo cercando la configurazione ottimale della rete e degli iperparametri, mentre il vero training-testing lo farò in un altro notebook.

In [23]:
s_type = 'max_vol'
results_path = os.path.join(base_path, 'results', f"results_{s_type}_no_sentiment.csv")
print(f"RESULTS path: {results_path}")

def objective_fn(space, **kwargs):
    return run_experiment(
        space,
        stock_type=s_type,
        max_epochs=MAX_EPOCHS,
        data=data_all[s_type],
        extremes=extremes_all[s_type]['95'],
        directions=directions_all[s_type],
        n_runs=5,
        verbose=0,
        results_path=results_path
    )

RESULTS path: /gdrive/My Drive/OptiRisk Thesis/experiments/11_final_experiment/results/results_max_vol_no_sentiment.csv


In [ ]:
optimizer_space = hp.choice('opt_name', [
    {
        'name': 'adam',
        'lr': hp.uniform('lr_adam', low=1e-5, high=1e-2)
    },
   {
       'name': 'adadelta',
   },
])

layer_space = {
    'num_layers': {
        'how_many': 1,
        'n_cells_1': 0, # hp.quniform('number_of_cells', low=10, high=100, q=2), # 96
        'n_cells_2': 0,
    },
#     hp.choice('number_of_layers', [
#         {
#             'how_many': 1,
#             'n_cells_1': 0, # hp.quniform('number_of_cells', low=10, high=100, q=2), # 96
#             'n_cells_2': 0,
#         },
#        {
#            'how_many': 2,
#            'n_cells_1': hp.quniform('number_of_cells_1', low=10, high=100, q=2),
#            'n_cells_2': hp.quniform('number_of_cells_2', low=10, high=50, q=2),
#        }
#     ]),
    'input_dropout': hp.uniform('dropout_kill_rate', low=0.0, high=0.4),
}

early_stop_space = {
    'patience': hp.quniform('early_stop_patience', low=5, high=25, q=1),
    'min_delta': hp.quniform('early_stop_min_delta', low=1e-4, high=1e-2, q=2e-4)
}

opt_space = {
    'optimizer': optimizer_space,
    'layers': layer_space,
    'bptt': hp.quniform('bptt_len', low=10, high=120, q=1),
    'early_stop': early_stop_space,
    'use_class_weight': False,
}

In [ ]:
print("COMINCIAMO\n")
n_trials = 50

load_trials = True
trials_filename = os.path.join(base_path, 'results', f"trials_{s_type}_no_sentiment.pickle")
best_filename = os.path.join(base_path, 'results', f"best_{s_type}_no_sentiment.pickle")

if load_trials:
    print("Loading trials from last execution")
    try:
        with open(trials_filename, 'rb') as infile:
            trials = pickle.load(infile)
            print(f"len(trials) = {len(trials)}")
    except FileNotFoundError:
        print("Trials file not found, starting with new trial object")
        trials = hy.Trials()
else:
    print("Starting new experiment with new trials object")
    trials = hy.Trials()


best = fmin(
    objective_fn,
    opt_space,
    algo=tpe.suggest,
    max_evals=len(trials) + n_trials,
    trials=trials
)

with open(trials_filename, 'wb') as outfile:
    pickle.dump(trials, outfile)

with open(best_filename, 'wb') as outfile:
    pickle.dump(best, outfile)

print("FINITO")

COMINCIAMO

Loading trials from last execution
Trials file not found, starting with new trial object
y[0] has 1954 elements of class 0.0
y[0] has 101 elements of class 1.0
y[1] has 1078 elements of class 0.0
y[1] has 977 elements of class 1.0
Loading result file at /gdrive/My Drive/OptiRisk Thesis/experiments/11_final_experiment/results/results_max_vol_no_sentiment.csv
{'bptt': 93.0, 'early_stop': {'min_delta': 0.0064, 'patience': 21.0}, 'layers': {'input_dropout': 0.2935786779396727, 'num_layers': {'how_many': 1, 'n_cells_1': 0, 'n_cells_2': 0}}, 'optimizer': {'lr': 0.004329954105583538, 'name': 'adam'}, 'use_class_weight': False}
Hyper parameters: bptt=93, optimizer=adam, learning_rate=0.004329954105583538
Iteration 0
Fitting model
Evaluating the model on the training set
Evaluating the model on the validation set
Evaluating the model on the test set
Scrivo i risultati
Iteration 1
Fitting model
Evaluating the model on the training set
Evaluating the model on the validation set
Evalua

## Scratch

Ora vedo come variare il threshold per ottenere le curve ROC

In [ ]:
def loss_function(theta, recall, fpr):
    """The loss function L = theta * (1 - recall) + (1 - theta) * fpr"""
    assert theta >= 0.0 and theta <= 1.0
    
    return theta * (1 - recall) + (1 - theta) * fpr


def utility_function(theta, loss):
    """The utility function U = min(theta, 1 - theta) - loss"""
    return min(theta, 1 - theta) - loss


def to_binary(prob: np.ndarray, thresh: float):
    assert thresh <= 1.0 and thresh >= 0.0
    
    return (prob >= thresh).astype(np.int8)


def recall_fpr_kss_precision(y_true, y_pred):
    """Compute recall, fpr and KSS score."""
    tp = np.sum(np.logical_and(y_true, y_pred))
    tn = np.sum(np.logical_and(
        np.logical_not(y_true),
        np.logical_not(y_pred)
    ))
    fp = np.sum(np.logical_and(
        np.logical_not(y_true),
        y_pred
    ))
    fn = np.sum(np.logical_and(
        y_true,
        np.logical_not(y_pred)
    ))
    
    recall = tp / (tp + fn)  # TP / (TP + FN)
    fpr = fp / (fp + tn)  # FP / (FP + TN)
    precision = tp / (tp + fp)
    
    kss = recall - fpr
    
    return recall, fpr, kss, precision

In [ ]:
w_t = np.arange(0, 1, 1e-3)
theta = 0.5

In [ ]:
def optimize_wt(w, theta, probabilities, y_true, verbose=False):
    """Get the best threshold for the class 1 probability."""
    recalls = np.zeros((w.shape[0], ), dtype=np.float64)
    fprs = copy.deepcopy(recalls)
    ksss = copy.deepcopy(recalls)
    precisions = copy.deepcopy(recalls)
    losses = copy.deepcopy(recalls)
    utilities = copy.deepcopy(recalls)

    for i, thresh in enumerate(w):
        if i % 200 == 0 and verbose:
            print(f"iteration {i} / {len(w_t)}")

        y_pred = to_binary(probabilities, thresh).astype(np.int8)
        recall, fpr, kss, precision = recall_fpr_kss_precision(y_true, y_pred)
        loss = loss_function(theta, recall, fpr)
        utility = utility_function(theta, loss)

        recalls[i] = recall
        precisions[i] = precision
        ksss[i] = kss
        fprs[i] = fpr
        losses[i] = loss
        utilities[i] = utility

    if verbose:
        print("Finished!")

    return recalls, fprs, ksss, precisions, losses, utilities

In [ ]:
probabilities_extreme_train = model.predict(X_train, batch_size=batch_size)[0][:, 1]
probabilities_up_down_train = model.predict(X_train, batch_size=batch_size)[1][:, 1]

probabilities_extreme_validation = model.predict(X_validation, batch_size=batch_size)[0][:, 1]
probabilities_up_down_validation = model.predict(X_validation, batch_size=batch_size)[1][:, 1]

# per le ROC sugli estremi
recalls_train, fprs_train, ksss_train, precisions_train, losses_train, utilities_train = \
optimize_wt(w_t, theta, probabilities_extreme_train, y_train[0].astype(np.int8))

recalls_validation, fprs_validation, ksss_validation, \
precisions_validation, losses_validation, utilities_validation = \
optimize_wt(w_t, theta, probabilities_extreme_validation, y_validation[0].astype(np.int8))

# per le ROC sul su-giù
recalls_train_ud, fprs_train_ud, ksss_train_ud, precisions_train_ud, losses_train_ud, utilities_train_ud = \
optimize_wt(w_t, theta, probabilities_up_down_train, y_train[1].astype(np.int8))

recalls_validation_ud, fprs_validation_ud, ksss_validation_ud, \
precisions_validation_ud, losses_validation_ud, utilities_validation_ud = \
optimize_wt(w_t, theta, probabilities_up_down_validation, y_validation[1].astype(np.int8))

In [ ]:
fig, ax = pl.subplots(nrows=1, ncols=2, figsize=(20, 10))
fig.suptitle(f"{s_type} stock", fontsize=16)

# primo plot: EXTREMES
# train set
i_sorted = np.argsort(fprs_train)

x_extreme = fprs_train[i_sorted]
y_extreme = recalls_train[i_sorted]
ax[0].plot(
    x_extreme,
    y_extreme,
    color='navy',
    label='train'
)

i_sweet = np.argmax(utilities_train)
best_x = fprs_train[i_sweet]
best_y = recalls_train[i_sweet]

ax[0].plot(
    best_x,
    best_y,
    marker='s',
    markersize=5,
    color='navy',
    label='train - best'
)

# validation set
i_sorted = np.argsort(fprs_validation)

x = fprs_validation[i_sorted]
y = recalls_validation[i_sorted]
ax[0].plot(
    x,
    y,
    color='forestgreen',
    label='validation'
)

i_sweet = np.argmax(utilities_validation)
best_x = fprs_validation[i_sweet]
best_y = recalls_validation[i_sweet]

ax[0].plot(
    best_x,
    best_y,
    marker='s',
    markersize=5,
    color='forestgreen',
    label='validation - best'
)

ax[0].set_title("Extreme prediction")

# ------------------------------------------------------- #
# secondo plot: UP-DOWN
# train set
i_sorted = np.argsort(fprs_train_ud)

x = fprs_train_ud[i_sorted]
y = recalls_train_ud[i_sorted]
ax[1].plot(
    x,
    y,
    color='navy',
    label='train'
)

i_sweet = np.argmax(utilities_train)
best_x = fprs_train_ud[i_sweet]
best_y = recalls_train_ud[i_sweet]

ax[1].plot(
    best_x,
    best_y,
    marker='s',
    markersize=5,
    color='navy',
    label='train - best'
)

# validation set
i_sorted = np.argsort(fprs_validation_ud)

x = fprs_validation_ud[i_sorted]
y = recalls_validation_ud[i_sorted]
ax[1].plot(
    x,
    y,
    color='forestgreen',
    label='validation'
)

i_sweet = np.argmax(utilities_validation)
best_x = fprs_validation_ud[i_sweet]
best_y = recalls_validation_ud[i_sweet]

ax[1].plot(
    best_x,
    best_y,
    marker='s',
    markersize=5,
    color='forestgreen',
    label='validation - best'
)

ax[1].set_title("Up-Down prediction")

# la linea del random classifier
for a in ax:
    a.plot([0, 1], [0, 1], color='black', linewidth=0.5)
    a.legend(loc='lower right', fontsize=14)
    a.set_xlim([0, 1.1])
    a.set_ylim([0, 1.1])
    a.set_xlabel('FPR', fontsize=16)
    a.set_ylabel('Recall', fontsize=16)

sns.despine()